### TODO

- voir si je peux utiliser la classification de Toufik afin de "nettoyer" les questions des structures récurrentes (voir classes Toufik). Une comparaison de similarité sur les termes médicaux seulement, où on unifie les termes médicaux de la question et de la réponse avant de faire des similarités par n-gram de caractères entre ces ensembles ?

- même idée, mais avec les phrases formées par Corina


### Ce qui a été fait

J'ai essayé de faire une première classification par règles. Pour chaque question, je crée des similarités cosinus au mot entre cette question et les autres questions et entre sa réponse et les autres réponses. Toujours pour chaque question, je ne garde que les résultats supérieurs à 0.8 (valeur à modifier par la suite ?). 

### Résultat

Comme on pouvait s'y attendre, on ne remarque pas grand chose. Les ensembles créés pour chaque question rassemblent souvent entre eux les questions avec des formes récurrentes (voir classes Toufik). Il faudrait pour chaque question indiquer à quelle classe elle appartient et, en fonction de la classe, extraire le segment contenant l'information médicale. 

Une fois que nous récupérons ce segment de la question, nous le plaçons dans une liste avec les réponses à la question, qui sont elles aussi des informations médicales. De là, on pourrait faire une similarité entre cet ensemble \[segment question / réponse\] et les autres ensembles \[segment question / réponse\].

### Remarques personnelles sur la tâche

Je distingue deux approches au problème :

- une approche visant à savoir si le jeu de données peut se suffire afin d'obtenir de meilleurs résultats que ceux obtenus dans le papier "source" : X questions nous donnent Y indices sur un concept médical, donc on a des liens entre mots clefs au sein du corpus qu'on exploite par la suite.

- une approche visant à créer ou exploiter une base de données médicale suffisement exhaustive afin de permettre une reconnaissance de termes clef entre eux : terme clef A de la question est plus souvent lié au terme B de la réponse X dans un contexte médical.


In [1]:
#Imports et fonctions

import json

#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosinus(q,liste_q):
    #vect = TfidfVectorizer()
    vect = CountVectorizer()
    liste_q_vect = vect.fit_transform(liste_q)
    doc_term_matrix = liste_q_vect.toarray()

    tgt_transform = vect.transform([q]).toarray()
    tgt_cosine = cosine_similarity(doc_term_matrix,tgt_transform)
    return tgt_cosine

def writeJson(path,data):
    with open(path,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4,ensure_ascii=False) #sans le ensure_ascii on a des erreurs d'encodage sur les accents / char speciaux / emojis sur Windows
        
def openJson(path):
    with open(path,'r',encoding='utf-8') as f:
        data = json.load(f)
    return data       

In [10]:
#Création d'un corpus de similarités entre questions et entre réponses aux questions (bonnes réponses uniquement)
"""
Similarité : cosinus
ce qui est comparé : une question avec toutes les autres questions du jeu de données
                     les réponses à une question avec toutes les réponses aux autres questions
ce qu'on obtient : un dictionnaire avec l'id d'une question Q1, le réponses à Q1, et 
un subdic avec l'id, les réponses et la similarité de chaque question par rapport à Q1
"""

import os
import pandas as pd

dic = {}
df = pd.read_csv("../data/csv/train.csv",delimiter=";")

ids = []
questions = []
reponses = []
total_reponses = []

for i in range(len(df)):
    ids.append(df["id"][i])
    questions.append(df["question"][i])
    reponses.append(" | ".join([df[f"answers.{l}"][i] for l in df["correct_answers"][i].split("|")]))
    total_reponses.append(df["correct_answers"][i])

for i,idd in enumerate(ids):
    dic[idd] = {"reponses":total_reponses[i],"similarités":{}}
    
    resQ = [list(s) for s in cosinus(questions[i],questions)]
    resR = [list(s) for s in cosinus(reponses[i],reponses)]
    
    for j,idd2 in enumerate(ids):
        dic[idd]["similarités"][idd2] = {"reponses":total_reponses[j],"simQ":resQ[j][0],"simR":resR[j][0]}

writeJson("sims.json",dic)

In [17]:
#Tri des questions par mesure de similarité

data = openJson("sims.json")

similarities_clusters = []

for idQ, subdic in data.items():
    sim_clus = []
    for k,v in subdic["similarités"].items():
        if v["simQ"] > 0.8:
            sim_clus.append(k)
    similarities_clusters.append(sim_clus)

similarities_clusters_clean = []
for l in similarities_clusters:
    if len(l) > 1:
        similarities_clusters_clean.append(l)
    
#on sauvegarde les clusters en supprimant les clusters similaires ET LES CLUSTERS D'UN ELEMENT
writeJson("cluster_by_question_similarities.json",[list(item) for item in set(tuple(row) for row in similarities_clusters_clean)]) 

In [4]:
import pandas as pd

df = pd.read_csv("../data/csv/train.csv",delimiter=";")
data = openJson("cluster_by_question_similarities.json")

questions_list = []
for l in data:
    new_l = []
    for i in range(len(df)):
        if df["id"][i] in l:
            new_l.append(df["question"][i])
    questions_list.append(new_l)
    
writeJson("questions.json",questions_list) 